# How to use the bounding box

Follow along this step-by-step guide to learn about the [`BoundingBox`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box).

To avoid any issues, run the cells in order and don't skip any cells.<br />
If something seems off, just restart the runtime and run the cells again.

# Install aviary

Install aviary in the current runtime using pip.

In [ ]:
! pip install -q geospaitial-lab-aviary

# Import aviary and verify the installation

In [ ]:
import aviary

print(aviary.__version__)

# Create a bounding box

A bounding box specifies the spatial extent of an area of interest.

You can pass the coordinates to the initializer of the
[`BoundingBox`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box).<br />
You can access the coordinates of the bounding box with the `x_min`, `y_min`, `x_max` and `y_max` attributes.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

We can visualize the bounding box with [folium](https://python-visualization.github.io/folium/latest) for a better understanding.

Install folium in the current runtime using pip.

In [ ]:
! pip install -q folium

We define a function `visualize_bounding_box`, so that we can reuse it in the next steps.

In [ ]:
import folium
import geopandas as gpd


def visualize_bounding_box(
    bounding_box: aviary.BoundingBox,
    zoom_start: int = 16,
) -> folium.Map:
    # Convert the bounding box to a geodataframe
    gdf = bounding_box.to_gdf(epsg_code=25832)

    # Compute the centroid of the bounding box
    centroid = gpd.GeoDataFrame(
        geometry=[gdf.union_all().centroid],
        crs=gdf.crs,
    )

    # Convert the centroid to EPSG:4326 (folium requires EPSG:4326)
    centroid_epsg_4326 = centroid.to_crs(epsg=4326)

    # Compute the location of the folium map
    location_epsg_4326 = [
        centroid_epsg_4326.geometry.y.mean(),
        centroid_epsg_4326.geometry.x.mean(),
    ]

    # Convert the bounding box to EPSG:4326 (folium requires EPSG:4326)
    gdf_epsg_4326 = gdf.to_crs(epsg=4326)

    # Create a folium map
    folium_map = folium.Map(
        location=location_epsg_4326,
        zoom_start=zoom_start,
        tiles=None,
    )

    # Add OpenStreetMap tiles to the folium map
    folium.TileLayer(
        tiles='OpenStreetMap',
        control=False,
    ).add_to(folium_map)

    # Add orthophotos layer to the folium map
    folium.raster_layers.WmsTileLayer(
        url='https://www.wms.nrw.de/geobasis/wms_nw_dop',
        layers='nw_dop_rgb',
        fmt='image/png',
        transparent=True,
        version='1.3.0',
        attr='<a href="https://www.bezreg-koeln.nrw.de/geobasis-nrw">Geobasis NRW</a>',
        name='Orthophotos',
        show=False,
    ).add_to(folium_map)

    # Define the style of the bounding box
    style_function = lambda feature: {
        'fillOpacity': .2,
        'color': 'black',
        'weight': 2,
    }

    # Add the bounding box to the folium map
    folium.GeoJson(
        data=gdf_epsg_4326,
        style_function=style_function,
        control=False,
    ).add_to(folium_map)

    # Add layer control to the folium map
    folium.LayerControl(
        collapsed=False,
    ).add_to(folium_map)

    return folium_map

Now we can visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

folium_map

You can set the coordinates of an already created bounding box with the `x_min`, `y_min`, `x_max` and `y_max` attributes.

In [ ]:
bounding_box.x_min = 363148
bounding_box.y_min = 5715390
bounding_box.x_max = 363276
bounding_box.y_max = 5715518

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

folium_map

A bounding box is an iterable object, so it supports indexing, length and iteration.

You can access the coordinates of the bounding box with the index operator.

In [ ]:
x_min = bounding_box[0]
y_min = bounding_box[1]
x_max = bounding_box[2]
y_max = bounding_box[3]

print(x_min)
print(y_min)
print(x_max)
print(y_max)

You can also unpack its coordinates.

In [ ]:
x_min, y_min, x_max, y_max = bounding_box

print(x_min)
print(y_min)
print(x_max)
print(y_max)

A bounding box has a length, which is obviously 4.

In [ ]:
print(len(bounding_box))

You can iterate over the coordinates of the bounding box.

In [ ]:
for coordinate in bounding_box:
    print(coordinate)

## Create a bounding box from a geodataframe

You can create a bounding box from a geodataframe with the
[`from_gdf`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box/#aviary.BoundingBox.from_gdf) class method.

In [ ]:
from shapely.geometry import box

gdf = gpd.GeoDataFrame(
    geometry=[box(363084, 5715326, 363340, 5715582)],
    crs='EPSG:25832',
)
bounding_box = aviary.BoundingBox.from_gdf(gdf=gdf)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

# Convert the geodataframe to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the geodataframe (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the geodataframe to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

The geodataframe may contain multiple polygons, e.g. the northern districts of Gelsenkirchen.

In [ ]:
url = (
    'https://raw.githubusercontent.com/geospaitial-lab/aviary/main'
    '/docs/how_to_guides/api/data/districts.geojson'
)
gdf = gpd.read_file(url)
bounding_box = aviary.BoundingBox.from_gdf(gdf=gdf)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(
    bounding_box=bounding_box,
    zoom_start=12,
)

# Convert the districts to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the districts (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the districts to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

# Buffer the bounding box

You can expand the bounding box with the
[`buffer`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box/#aviary.BoundingBox.buffer) method.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

In [ ]:
buffered_bounding_box = bounding_box.buffer(buffer_size=64)

print(buffered_bounding_box.x_min)
print(buffered_bounding_box.y_min)
print(buffered_bounding_box.x_max)
print(buffered_bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=buffered_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the original bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

You can also shrink the bounding box with the
[`buffer`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box/#aviary.BoundingBox.buffer) method.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

In [ ]:
buffered_bounding_box = bounding_box.buffer(buffer_size=-64)

print(buffered_bounding_box.x_min)
print(buffered_bounding_box.y_min)
print(buffered_bounding_box.x_max)
print(buffered_bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=buffered_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the original bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

# Quantize the bounding box

You can align the bounding box to a grid with the
[`quantize`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box/#aviary.BoundingBox.quantize) method.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

In [ ]:
quantized_bounding_box = bounding_box.quantize(value=128)

print(quantized_bounding_box.x_min)
print(quantized_bounding_box.y_min)
print(quantized_bounding_box.x_max)
print(quantized_bounding_box.y_max)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=quantized_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the original bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

# Convert the bounding box to a geodataframe

You can convert the bounding box to a geodataframe with the
[`to_gdf`](https://geospaitial-lab.github.io/aviary/api_reference/bounding_box/#aviary.BoundingBox.to_gdf) method.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box.x_min)
print(bounding_box.y_min)
print(bounding_box.x_max)
print(bounding_box.y_max)

In [ ]:
gdf = bounding_box.to_gdf(epsg_code=25832)

print(gdf)